## KarpovCourses. Продуктовая аналитика 
### Сегментация клиентов в Python 
 
### RFM-анализ

In [1]:
import pandas as pd
import numpy as np


# Matplotlib forms basis for visualization in Python
import matplotlib.pyplot as plt

# We will use the Seaborn library
import seaborn as sns
sns.set()

# Graphics in SVG format are more sharp and legible
get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'svg'")

# Increase the default plot size and set the color scheme
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['image.cmap'] = 'viridis'

In [6]:
orders = pd.read_csv('RFM_ht_data.csv',sep=',', low_memory=False)

In [12]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332730 entries, 0 to 332729
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     332730 non-null  object        
 1   CustomerCode  332730 non-null  object        
 2   InvoiceDate   332730 non-null  datetime64[ns]
 3   Amount        332730 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 10.2+ MB


In [10]:
orders.head(5)

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [11]:
orders['InvoiceDate'] = pd.to_datetime(orders['InvoiceDate'])

In [13]:
last_date = orders['InvoiceDate'].max()

In [14]:
last_date

Timestamp('2020-09-30 00:00:00')

In [15]:
rfmTable = orders.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (last_date - x.max()).days, # Recency #Количество дней с последнего заказа
                                        'InvoiceNo': lambda x: len(x),      # Frequency #Количество заказов
                                        'Amount': lambda x: x.sum()}) # Monetary Value #Общая сумма по всем заказам

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)

In [19]:
rfmSegmentation = rfmTable

In [20]:
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4


def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1

In [24]:
quantiles = rfmTable.quantile(q=[0.25, 0.5, 0.75])

In [25]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles))

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) + rfmSegmentation.F_Quartile.map(str) + rfmSegmentation.M_Quartile.map(str)

In [29]:
rfmSegmentation.head(5)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443


Какое максимальное кол-во покупок было совершено одним пользователем?

In [18]:
rfmTable.frequency.max()

204

Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)

In [27]:
rfmSegmentation.query("M_Quartile==4").monetary_value.max()

765.0

Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?

In [31]:
rfmSegmentation.query("F_Quartile==1").frequency.min()

4

Какое максимальное количество дней может пройти с момента последней покупки для того, чтобы пользователь попал в класс 2 в подсегменте R?

In [32]:
rfmSegmentation.query("R_Quartile==2").recency.max()

8

In [41]:
grRFM = rfmSegmentation.groupby("RFMClass", as_index=False).agg(cnt_rfm=('RFMClass', 'count'))

Сколько пользователей попало в сегмент 111?

In [43]:
grRFM.query("RFMClass=='111'")

,RFMClass,cnt_rfm
0,111,9705


Сколько пользователей попало в сегмент 311?

In [44]:
grRFM.query("RFMClass=='311'")

,RFMClass,cnt_rfm
32,311,1609


В каком RFM-сегменте самое большое кол-во пользователей?

In [46]:
grRFM.sort_values("cnt_rfm", ascending=False).head(1)

,RFMClass,cnt_rfm
63,444,10624


В каком RFM-сегменте самое маленькое кол-во пользователей?
Какое количество пользователей попало в самый малочисленный сегмент?

In [47]:
grRFM.sort_values("cnt_rfm").head(1)

,RFMClass,cnt_rfm
51,414,2
